<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/BanglaSER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!gdown --id 1fZoWnL0neesTfj3l1kQRNN5O0Sfm3Bjl
time.sleep(5)

!unzip /content/BanglaSER.zip

In [9]:
import os
import shutil
import random


dataset_folder = "/content/t9h6p943xy-5/BanglaSER/"
train_folder = "/content/train_set"
test_folder = "/content/test_set"


os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)


actor_folders = [f for f in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, f))]


for actor_folder in actor_folders:
    actor_path = os.path.join(dataset_folder, actor_folder)


    audio_files = [f for f in os.listdir(actor_path) if f.endswith('.wav')]  # Assuming audio files are .wav


    random.shuffle(audio_files)


    split_idx = int(0.8 * len(audio_files))
    train_files = audio_files[:split_idx]
    test_files = audio_files[split_idx:]


    os.makedirs(os.path.join(train_folder, actor_folder), exist_ok=True)
    os.makedirs(os.path.join(test_folder, actor_folder), exist_ok=True)


    for file in train_files:
        src = os.path.join(actor_path, file)
        dest = os.path.join(train_folder, actor_folder, file)
        shutil.copy(src, dest)

    for file in test_files:
        src = os.path.join(actor_path, file)
        dest = os.path.join(test_folder, actor_folder, file)
        shutil.copy(src, dest)


print("Dataset Split Completed:")
for actor_folder in actor_folders:
    train_count = len(os.listdir(os.path.join(train_folder, actor_folder)))
    test_count = len(os.listdir(os.path.join(test_folder, actor_folder)))
    print(f"Actor '{actor_folder}': Train = {train_count}, Test = {test_count}")


Dataset Split Completed:
Actor 'Actor 16': Train = 28, Test = 8
Actor 'Actor 24': Train = 36, Test = 9
Actor 'Actor 19': Train = 36, Test = 9
Actor 'Actor 18': Train = 28, Test = 8
Actor 'Actor 11': Train = 28, Test = 8
Actor 'Actor 17': Train = 36, Test = 9
Actor 'Actor 14': Train = 36, Test = 9
Actor 'Actor 28': Train = 36, Test = 9
Actor 'Actor 33': Train = 36, Test = 9
Actor 'Actor 29': Train = 28, Test = 8
Actor 'Actor 21': Train = 36, Test = 9
Actor 'Actor 26': Train = 36, Test = 9
Actor 'Actor 05': Train = 36, Test = 9
Actor 'Actor 20': Train = 28, Test = 8
Actor 'Actor 22': Train = 36, Test = 9
Actor 'Actor 10': Train = 36, Test = 9
Actor 'Actor 12': Train = 28, Test = 8
Actor 'Actor 13': Train = 28, Test = 8
Actor 'Actor 02': Train = 36, Test = 9
Actor 'Actor 34': Train = 36, Test = 9
Actor 'Actor 23': Train = 36, Test = 9
Actor 'Actor 08': Train = 36, Test = 9
Actor 'Actor 15': Train = 36, Test = 9
Actor 'Actor 32': Train = 36, Test = 9
Actor 'Actor 01': Train = 36, Test = 9


In [10]:
import os

# Define the path to the train folder
train_folder = 'train_set'  # Change this to your actual path if it's different

# Step 1: List all actor subfolders inside the train folder
actor_folders = [f for f in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, f))]

# Step 2: Print out the content of each actor subfolder (i.e., audio files)
for actor_folder in actor_folders:
    actor_path = os.path.join(train_folder, actor_folder)

    # List all audio files in the current actor folder (assuming they're '.wav' files)
    audio_files = [f for f in os.listdir(actor_path) if f.endswith('.wav')]  # Change if you have different file types

    print(f"Actor Folder: {actor_folder}")
    print(f"Audio Files: {audio_files}")
    print("-" * 40)  # Separator between actor's files


Actor Folder: Actor 16
Audio Files: ['03-01-03-02-02-01-16.wav', '03-01-04-02-03-01-16.wav', '03-01-01-01-02-01-16.wav', '03-01-02-01-02-01-16.wav', '03-01-03-02-01-02-16.wav', '03-01-01-01-01-01-16.wav', '03-01-03-02-03-02-16.wav', '03-01-04-02-01-01-16.wav', '03-01-02-01-01-03-16.wav', '03-01-02-01-02-03-16.wav', '03-01-03-02-02-02-16.wav', '03-01-01-01-01-02-16.wav', '03-01-03-02-02-03-16.wav', '03-01-02-01-03-02-16.wav', '03-01-02-01-02-02-16.wav', '03-01-02-01-03-03-16.wav', '03-01-01-01-03-01-16.wav', '03-01-01-01-01-03-16.wav', '03-01-01-01-02-03-16.wav', '03-01-04-02-01-02-16.wav', '03-01-03-02-03-01-16.wav', '03-01-04-02-02-02-16.wav', '03-01-04-02-03-03-16.wav', '03-01-01-01-02-02-16.wav', '03-01-01-01-03-02-16.wav', '03-01-02-01-03-01-16.wav', '03-01-03-02-03-03-16.wav', '03-01-04-02-01-03-16.wav']
----------------------------------------
Actor Folder: Actor 24
Audio Files: ['03-01-05-01-01-03-24.wav', '03-01-03-02-02-03-24.wav', '03-01-03-02-02-02-24.wav', '03-01-03-02-03-0

In [11]:
import os


test_folder = 'test_set'


actor_folders = [f for f in os.listdir(test_folder) if os.path.isdir(os.path.join(test_folder, f))]


for actor_folder in actor_folders:
    actor_path = os.path.join(test_folder, actor_folder)


    audio_files = [f for f in os.listdir(actor_path) if f.endswith('.wav')]

    print(f"Actor Folder: {actor_folder}")
    print(f"Audio Files: {audio_files}")
    print("-" * 40)


Actor Folder: Actor 16
Audio Files: ['03-01-04-02-03-02-16.wav', '03-01-04-02-02-03-16.wav', '03-01-04-02-02-01-16.wav', '03-01-02-01-01-01-16.wav', '03-01-03-02-01-03-16.wav', '03-01-01-01-03-03-16.wav', '03-01-03-02-01-01-16.wav', '03-01-02-01-01-02-16.wav']
----------------------------------------
Actor Folder: Actor 24
Audio Files: ['03-01-01-01-03-02-24.wav', '03-01-01-01-03-03-24.wav', '03-01-01-01-02-03-24.wav', '03-01-01-01-01-01-24.wav', '03-01-02-01-03-01-24.wav', '03-01-05-01-01-02-24.wav', '03-01-02-01-02-01-24.wav', '03-01-01-01-02-02-24.wav', '03-01-04-02-01-03-24.wav']
----------------------------------------
Actor Folder: Actor 19
Audio Files: ['03-01-03-02-03-03-19.wav', '03-01-03-02-03-01-19.wav', '03-01-01-01-03-03-19.wav', '03-01-04-02-01-02-19.wav', '03-01-04-02-02-02-19.wav', '03-01-02-01-02-03-19.wav', '03-01-02-01-01-02-19.wav', '03-01-01-01-03-02-19.wav', '03-01-05-01-02-02-19.wav']
----------------------------------------
Actor Folder: Actor 18
Audio Files: ['